Скачивание и парсинг данных о книгах и читателях с сайта www.livelib.ru

In [64]:
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from html.parser import HTMLParser
import string
from selenium import webdriver
import html
from time import sleep
from selenium.common.exceptions import NoSuchElementException
from pandas import DataFrame
import csv

In [59]:
way_driver = "D:\Program\chromedriver.exe"
syte_url = "https://www.livelib.ru"
result = []
all_books = []
number_all_books = 0

In [69]:
#Здесь список readers[] будет заполняться логинами из csv файла

In [60]:
def get_book_info(href, number):
    book_info = {
        'href': '-',
        'number': 0,
    }
    book_info['href'] = href
    book_info['number'] = number
    return book_info

In [61]:
def get_reader_info(driver, reader_name, t_sleep=1):
    number_marks_book = 0
    reader_url = syte_url+"/reader/"+reader_name
    reader_html = requests.get(reader_url).content
    soup = BeautifulSoup(reader_html, 'html.parser')
    driver.get(reader_url)
    sleep(t_sleep)
    reader_info = {
        'login': '0',
        'gender': '-',
        'rating': '0',
        'marks_books': [],
    }
    reader_info['login'] = reader_name
    items = driver.find_elements_by_class_name("group-row-title")
    for item in items:
        s = item.text
        if s.find('Пол:') == 0:
            reader_info['gender'] = s[5:len(s)]
        elif s.find('Рейтинг:') == 0:
            reader_info['rating'] = s[9:len(s)]
    sleep(t_sleep)
    #Прочитанные книги
    reader_url = syte_url+"/reader/"+reader_name+"/read"
    reader_html = requests.get(reader_url).content
    soup = BeautifulSoup(reader_html, 'html.parser')
    driver.get(reader_url)
    sleep(t_sleep)
    number_item = driver.find_element_by_xpath("//*[@class='column-670 subcontainer on-left']")
    s = number_item.text
    s_number = ''
    i = 0
    while (s[i] < '0') or (s[i] > '9'):
        i = i + 1
    while (s[i] >= '0') and (s[i] <= '9'):
        s_number = s_number + s[i]
        i = i + 1
    number = int(s_number)
    number_page = number // 20
    if number % 20:
        number_page = number_page + 1
    for i in range(1, number_page + 1):
        reader_url = syte_url+"/reader/"+reader_name+"/read/~"+str(i)
        reader_html = requests.get(reader_url).content
        soup = BeautifulSoup(reader_html, 'html.parser')
        driver.get(reader_url)
        sleep(t_sleep)
        items_books = driver.find_elements_by_xpath("//*[@class='brow-book-name with-cycle']")
        items_marks = driver.find_elements_by_class_name("brow-ratings")
        for item_book, item_mark in zip(items_books, items_marks):
            mark_book = {
                'href': '',
                'mark': '',
                'number': 0,
            }
            mark_book['number'] = number_marks_book
            number_marks_book = number_marks_book + 1
            mark_book['href'] = item_book.get_attribute("href")
            s = item_mark.text
            s_mark = ''
            i = 0
            while (s[i] < '0') or (s[i] > '9'):
                i = i + 1
            while (s[i] >= '0') and (s[i] <= '9'):
                s_mark = s_mark + s[i]
                i = i + 1
            if s[i] == '.':
                s_mark = s_mark + s[i]
                i = i + 1
                while (s[i] >= '0') and (s[i] <= '9'):
                    s_mark = s_mark + s[i]
                    i = i + 1
            mark_book['mark'] = s_mark
            reader_info['marks_books'].append(mark_book)
    #Рецензии
    reader_url = syte_url+"/reader/"+reader_name+"/reviews"
    reader_html = requests.get(reader_url).content
    soup = BeautifulSoup(reader_html, 'html.parser')
    driver.get(reader_url)
    sleep(t_sleep)
    number_item = driver.find_element_by_xpath('//*[@style="margin-right:50px;"]')
    s = number_item.text
    s_number = ''
    i = 0
    while (s[i] < '0') or (s[i] > '9'):
        i = i + 1
    while (s[i] >= '0') and (s[i] <= '9'):
        s_number = s_number + s[i]
        i = i + 1
    number = int(s_number)
    number_page = number // 25
    if number % 25:
        number_page = number_page + 1
    for i in range(1, number_page + 1):
        reader_url = syte_url+"/reader/"+reader_name+"/reviews/~"+str(i)
        reader_html = requests.get(reader_url).content
        soup = BeautifulSoup(reader_html, 'html.parser')
        driver.get(reader_url)
        sleep(t_sleep)
        items_books = driver.find_elements_by_class_name("book_name")
        items_marks = driver.find_elements_by_class_name("rating-book")
        for item_book, item_mark in zip(items_books, items_marks):
            mark_book = {
                'href': '',
                'mark': '',
                'number': 0,
            }
            mark_book['number'] = number_marks_book
            number_marks_book = number_marks_book + 1
            mark_book['href'] = item_book.get_attribute("href")
            flag = true
            mark_book['mark'] = item_mark.text
            for j in reader_info['marks_books']:
                if j['href'] == mark_book['href']:
                    flag = false
            if flag:
                reader_info['marks_books'].append(mark_book)
    return reader_info

In [62]:
readers = ['vovanthik', 'aurora_sky']
with webdriver.Chrome(way_driver) as driver:
    for reader in readers:
        r_i = get_reader_info(driver, reader, 2)
        for i in all_books:
            flag = True
            for j in r_i['marks_books']:
                if (j == 'href') or (j == 'mark') or (j == 'number'):
                    flag = False
                elif i['href'] == j['href']:
                    flag = False
            if flag:
                mark_book = {
                    'href': '',
                    'mark': '',
                    'number': 0,
                }
                mark_book['number'] = i['number']
                mark_book['href'] = i['href']
                mark_book['mark'] = '0'
                r_i['marks_books'].append(mark_book)
        for i in r_i['marks_books']:
            flag = True
            for j in all_books:
                if (i == 'href') or (i == 'mark') or (i == 'number'):
                    flag = False
                elif i['href'] == j['href']:
                    flag = False
            if flag:
                all_books.append(get_book_info(i['href'], number_all_books))
                mark_book = {
                    'href': '',
                    'mark': '',
                    'number': 0,
                }
                mark_book['number'] = number_all_books
                i['number'] = number_all_books
                number_all_books = number_all_books + 1
                mark_book['href'] = i['href']
                mark_book['mark'] = '0'
                for j in result:
                    j['marks_books'].append(mark_book)
        result.append(r_i)

In [63]:
from pandas import DataFrame
books = ['Readers/Books']
for i in all_books:
    books.append(i['href'])
marks = []
for r_i in result:
    one_marks = [r_i['login']]
    for i in range(number_all_books):
        for b_i in r_i['marks_books']:
            if (b_i != 'href') and (b_i != 'mark') and (b_i != 'number'):
                if b_i['number'] == i:
                    one_marks.append(b_i['mark'])    
    marks.append(one_marks)
df = DataFrame(marks, columns=books)
df.to_excel('marks.xlsx', sheet_name='sheet1', index=False)